In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def multitime_to_pd(list):
    df = pd.DataFrame(columns = ['Mean', 'Std.Dev.', 'Min', 'Median', 'Max'], index = ['real', 'user', 'sys'])
    for i in range (1,4):
        row = re.sub(' +', ' ', list[i]).split(" ")[:6]
        df.loc[row[0]] = row[1:]        
    return df

def plot_times(df):
    ax = df.plot(x='name', y='time', kind='bar')
    ax.set_xlabel(None)
    ax.set_ylabel('Execution time (s)')
    ax.set_title('Execution Time Comparison')
    for i, v in enumerate(df['time']):
        ax.text(i, v, str(v), ha='center', va='bottom', rotation=90)
    plt.show()
    
    
def plot_grouped(groups, times):
    # place group labels
    x = np.arange(len(groups))  # the label locations
    width = 0.15  # the width of the bars
    offset = -0.15

    fig, ax = plt.subplots()

    for attribute, measurement in times.items():
        rects = ax.bar(x + offset, measurement, width, label=attribute)
        ax.bar_label(rects, padding=3, rotation=90)
        offset += width

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Execution time (s)')
    ax.set_title('Execution Time Comparison')
    ax.set_xticks(x + 0.1, groups)
    ax.legend(loc='upper left', ncol=3)
    #ax.set_ylim(0, 6)

    plt.show()

In [3]:
dfCsmall=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Small_dataset/C/time.txt", "r").readlines())
dfCmedium=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Medium_dataset/C/time.txt", "r").readlines())
dfClarge=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Large_dataset/C/time.txt", "r").readlines())

dfWASMsmall=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Small_dataset/WASM/time.txt", "r").readlines())
dfWASMmedium=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Medium_dataset/WASM/time.txt", "r").readlines())
dfWASMlarge=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Large_dataset/WASM/time.txt", "r").readlines())

dfJSsmall=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Small_dataset/WASM+JS/time.txt", "r").readlines())
dfJSmedium=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Medium_dataset/WASM+JS/time.txt", "r").readlines())
dfJSlarge=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Large_dataset/WASM+JS/time.txt", "r").readlines())

dfDockerCsmall=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Small_dataset/Docker+C/time.txt", "r").readlines())
dfDockerCmedium=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Medium_dataset/Docker+C/time.txt", "r").readlines())
dfDockerClarge=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Large_dataset/Docker+C/time.txt", "r").readlines())

dfDockerRuntimesmall=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Small_dataset/Docker+runtime/time.txt", "r").readlines())
dfDockerRuntimemedium=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Medium_dataset/Docker+runtime/time.txt", "r").readlines())
dfDockerRuntimelarge=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Large_dataset/Docker+runtime/time.txt", "r").readlines())

dfDockerWASMsmall=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Small_dataset/Docker+WASM/time.txt", "r").readlines())
dfDockerWASMmedium=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Medium_dataset/Docker+WASM/time.txt", "r").readlines())
dfDockerWASMlarge=multitime_to_pd(open("./Benchmarks/Caesar-cipher/Large_dataset/Docker+WASM/time.txt", "r").readlines())

FileNotFoundError: [Errno 2] No such file or directory: './Benchmarks/Caesar-cipher/Small_dataset/C/time.txt'

In [ ]:
# Extract the execution time values that you want to plot
name_values = ['Csmall', 'Cmedium', 'Clarge', 
               'WASMsmall', 'WASMmedium', 'WASMlarge', 
               'JSsmall', 'JSmedium', 'JSlarge', 
               'DockerCsmall', 'DockerCmedium', 'DockerClarge',
               'DockerRuntimesmall', 'DockerRuntimemedium', 'DockerRuntimelarge',
               'DockerWASMsmall', 'DockerWASMmedium', 'DockerWASMlarge',
              ]

time_values = [float(dfCsmall.loc['real', 'Mean']), float(dfCmedium.loc['real', 'Mean']), float(dfClarge.loc['real', 'Mean']), 
               float(dfWASMsmall.loc['real', 'Mean']), float(dfWASMmedium.loc['real', 'Mean']), float(dfWASMlarge.loc['real', 'Mean']),
               float(dfJSsmall.loc['real', 'Mean']), float(dfJSmedium.loc['real', 'Mean']), float(dfJSlarge.loc['real', 'Mean']),
               float(dfDockerCsmall.loc['real', 'Mean']), float(dfDockerCmedium.loc['real', 'Mean']), float(dfDockerClarge.loc['real', 'Mean']),
               float(dfDockerRuntimesmall.loc['real', 'Mean']), float(dfDockerRuntimemedium.loc['real', 'Mean']), float(dfDockerRuntimelarge.loc['real', 'Mean']),
               float(dfDockerWASMsmall.loc['real', 'Mean']), float(dfDockerWASMmedium.loc['real', 'Mean']), float(dfDockerWASMlarge.loc['real', 'Mean']),
              ]

# Create a new dataframe to store the extracted values
df = pd.DataFrame({'name': name_values, 'time': time_values})
df

In [ ]:
groups = ("Small", "Medium", "Large")
times = {
    'C': (float(dfCsmall.loc['real', 'Mean']), float(dfCmedium.loc['real', 'Mean']), float(dfClarge.loc['real', 'Mean'])),
    'WASM': (float(dfWASMsmall.loc['real', 'Mean']), float(dfWASMmedium.loc['real', 'Mean']), float(dfWASMlarge.loc['real', 'Mean'])),
    'Docker+C': (float(dfDockerCsmall.loc['real', 'Mean']), float(dfDockerCmedium.loc['real', 'Mean']), float(dfDockerClarge.loc['real', 'Mean'])),
    'Docker+WASM': (float(dfDockerWASMsmall.loc['real', 'Mean']), float(dfDockerWASMmedium.loc['real', 'Mean']), float(dfDockerWASMlarge.loc['real', 'Mean'])),
    'StandardDocker+Runtime': (float(dfDockerRuntimesmall.loc['real', 'Mean']), float(dfDockerRuntimemedium.loc['real', 'Mean']), float(dfDockerRuntimelarge.loc['real', 'Mean'])),
    'WASM+JS': (float(dfJSsmall.loc['real', 'Mean']), float(dfJSmedium.loc['real', 'Mean']), float(dfJSlarge.loc['real', 'Mean'])),
    }

plot_times(df)

plot_grouped(groups, times)


In [4]:
!ls

bench.ipynb  Benchmarks  README.md
